1.利用指定列对数据进行分组

In [2]:
import numpy as np
import pandas as pd
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),'data2' : np.random.randn(5)})

In [3]:
df

,key1,key2,data1,data2
0,a,one,0.170342,-0.830886
1,a,two,-0.684807,-0.313282
2,b,one,-0.560251,-0.235328
3,b,two,0.785432,0.712333
4,a,one,1.403779,3.060934


In [6]:
df.groupby('key1').mean()  #这里默认去掉key2非数据的列

,data1,data2
key1,,
a,0.296438,0.638922
b,0.112590,0.238502


In [12]:
df.groupby('key2').size()

key2
one    3
two    2
dtype: int64

2.产生的分组对象是可迭代对象

In [13]:
for (k1,k2),group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.170342 -0.830886
4    a  one  1.403779  3.060934
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.684807 -0.313282
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.560251 -0.235328
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.785432  0.712333


In [14]:
pieces = dict(list(df.groupby('key1')))  #生成字典
pieces

{'a':   key1 key2     data1     data2
 0    a  one  0.170342 -0.830886
 1    a  two -0.684807 -0.313282
 4    a  one  1.403779  3.060934, 'b':   key1 key2     data1     data2
 2    b  one -0.560251 -0.235328
 3    b  two  0.785432  0.712333}

3.分组后已相当于另一个Dataframe数据表，可以直接索引操作

In [23]:
df.groupby(['key1', 'key2'])[['data2']].mean()   #传入列表输入dataframe，传入变量名输入serise

data2
key1 key2          
a    one   1.115024
     two  -0.313282
b    one  -0.235328
     two   0.712333

4.分组也可以传入字典和serise，函数

5.分组可以很灵活的使用内置函数，传入函数操作

In [24]:
df.groupby('key1').describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.296438  1.049987 -0.684807 -0.257232  0.170342  0.787061   
b      2.0  0.112590  0.951541 -0.560251 -0.223830  0.112590  0.449011   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.403779   3.0  0.638922  2.113430 -0.830886 -0.572084 -0.313282   
b     0.785432   2.0  0.238502  0.670098 -0.235328  0.001587  0.238502   

                          
           75%       max  
key1                      
a     1.373826  3.060934  
b     0.475418  0.712333

In [27]:
def fun(arr):
    return arr.max() - arr.min()
df.groupby('key2').agg(fun)

,data1,data2
key2,,
one,1.964030,3.891820
two,1.470239,1.025615


6.分组后对列进行函数操作是非常灵活的

In [30]:
df_group = df.groupby('key2')
df_group.agg({'data1':'mean','data2':'count'})  #对data1数据却平均，对data2求个数

,data1,data2
key2,,
one,0.337957,3
two,0.050312,2


In [32]:
df_group['data1','data2'].agg([('the mean','mean'),('the max','max')])

data1               data2          
      the mean   the max  the mean   the max
key2                                        
one   0.337957  1.403779  0.664906  3.060934
two   0.050312  0.785432  0.199526  0.712333

In [36]:
df_group = df.groupby('key2',as_index=False) #不要索引
df_group.mean()

,key2,data1,data2
0,one,0.337957,0.664906
1,two,0.050312,0.199526


7.groupby和cut和qcut结合

In [37]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
grouped = frame.data2.groupby(quartiles)

In [41]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-2.986, -1.478]",68.0,2.837534,0.125067,-2.494835
"(-1.478, 0.0242]",434.0,2.946924,-0.006377,-2.548260
"(0.0242, 1.526]",441.0,2.924476,0.014448,-2.656385
"(1.526, 3.028]",57.0,1.807267,0.037184,-3.272901


In [46]:
grouping = pd.qcut(frame.data1,10)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-2.981, -1.283]",100.0,2.837534,0.100073,-2.494835
"(-1.283, -0.836]",100.0,2.946924,-0.008180,-2.191039
"(-0.836, -0.513]",100.0,2.544819,-0.152163,-2.417337
"(-0.513, -0.248]",100.0,2.086581,0.145434,-2.548260
"(-0.248, 0.0228]",100.0,2.799733,-0.011089,-2.345486
"(0.0228, 0.275]",100.0,2.201994,0.118914,-2.320934
"(0.275, 0.519]",100.0,2.924476,-0.061671,-2.526671
"(0.519, 0.859]",100.0,2.130321,-0.073017,-2.656385
"(0.859, 1.28]",100.0,2.590781,0.119542,-2.108453


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
key1     5 non-null object
key2     5 non-null object
data1    5 non-null float64
data2    5 non-null float64
dtypes: float64(2), object(2)
memory usage: 240.0+ bytes


pivot table执行透视表
crosstab交叉表，计算数据频率的